## Laboration Statistiska metoder

### Introduktion

Jag kommer i denna labb göra en linjär regression på ett dataset samt tolka datan statistiskt.

### Metod

Jag har skapat en regressionsklass i Linear_regression.py, som hanterar alla beräkningar. I denna fil kommer jag kalla på klassens olika funktioner och visa resultaten. Datasetet som kommer utforskas är 'housing.csv', ett känt dataset som innehåller data om huspriser från Kalifornien 1997. Jag kommer göra två regressionsanalyser, en med endast numeriska kategorier och en där jag implementerar den kategoriska datan. Jag kommer även göra en 'one-hot encode' på den kategoriska datan, och lägga till denna i min regression för att sedan göra min analys. Jag har gjort en summeringsfunktion som används i slutet för att visa resultaten, med inspiration från summeringsfunktioner som finns implementerade i exempelvis statsmodels och liknande bibliotek.
Innan detta görs gör jag en utforskning av datasetet, samt en enkel rensning. 

In [14]:
import numpy as np
import scipy.stats as st
import pandas as pd
from pandas.api import types as ptypes
from Linear_regression import LinearRegression

df = pd.read_csv("../Data/housing.csv")
display(df.head())
is_num = df.dtypes.apply(ptypes.is_numeric_dtype)
numeric = is_num.sum()
categorical = len(is_num) - numeric

print(f'Antal kolumner: {df.columns.size} \nAntal numeriska kolumner: {numeric} \nAntal kategoriska kolumner: {categorical}')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Antal kolumner: 10 
Antal numeriska kolumner: 9 
Antal kategoriska kolumner: 1


In [15]:
display(df.isna().sum())
df_clean = df.dropna()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Datasetet består av 10 kolumner, varav 9 är numeriska och 1 är kategorisk, där den kategoriska är ocean_proximity. Det finns 207 NaN-värden i datasetet, alla befinner sig i kolumnen 'total_bedrooms'. Jag väljet att droppa dessa vilket ger mig 207 observationer mindre i mitt slutgiltiga dataset. 

In [16]:
df = df_clean.copy()
df['ocean_proximity'].value_counts()

ocean_proximity
<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: count, dtype: int64

Värt att notera här är att 'ISLAND' endast förekommer 5 gånger i datasetet, vilket framöver kommer göra den statistiskt opålitlig, och skapar även en obalans bland kategorierna. Jag väljer därför att slå ihop 'ISLAND' med 'NEAR OCEAN', då jag anser det vara ett geografiskt rimligt val. Jag kommer använda '<1H OCEAN' som min **baseline** då det är den mest förekommande kategorin.

In [17]:
df['ocean_proximity'] = df['ocean_proximity'].replace({'ISLAND': 'NEAR OCEAN'})
df['ocean_proximity'].value_counts()

ocean_proximity
<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2633
NEAR BAY      2270
Name: count, dtype: int64

## Första modellen

För att bygga min första regressionsmodell började jag med att analysera korrelationerna mellan de numeriska variablerna i datasetet med hjälp av min pearsonmetod. Syftet var att identifiera multikollinearitet — alltså variabler som i praktiken mäter samma sak och därför riskerar att skapa instabilitet i modellen. Jag väljer 'median_house_value' som min målvariabel, och resterande numeriska kategorier i min X-matris, som jag sedan skapar en korrelationsmatris av.

In [18]:
X = df_clean.iloc[:, :9].drop(columns=['median_house_value'], errors='ignore')
y = df_clean['median_house_value']

model = LinearRegression()
model.fit(X,y)

r = model.pearson_corr()
feature_names = df_clean.columns[:8]

corr_df = pd.DataFrame(r, columns=feature_names, index=feature_names)
corr_df


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
longitude,1.000000,-0.924616,-0.109357,0.045480,0.069608,0.100270,0.056513,-0.015550
latitude,-0.924616,1.000000,0.011899,-0.036667,-0.066983,-0.108997,-0.071774,-0.079626
housing_median_age,-0.109357,0.011899,1.000000,-0.360628,-0.320451,-0.295787,-0.302768,-0.118278
total_rooms,0.045480,-0.036667,-0.360628,1.000000,0.930380,0.857281,0.918992,0.197882
total_bedrooms,0.069608,-0.066983,-0.320451,0.930380,1.000000,0.877747,0.979728,-0.007723
population,0.100270,-0.108997,-0.295787,0.857281,0.877747,1.000000,0.907186,0.005087
households,0.056513,-0.071774,-0.302768,0.918992,0.979728,0.907186,1.000000,0.013434
median_income,-0.015550,-0.079626,-0.118278,0.197882,-0.007723,0.005087,0.013434,1.000000


In [19]:
corr = df_clean.iloc[:, :8].corr()

high_corr = (corr.abs().where(np.triu(np.ones(corr.shape), k=1).astype(bool)).stack())

high_corr = high_corr[high_corr > 0.8]
high_corr

longitude       latitude          0.924616
total_rooms     total_bedrooms    0.930380
                population        0.857281
                households        0.918992
total_bedrooms  population        0.877747
                households        0.979728
population      households        0.907186
dtype: float64

Analysen visade att **total_rooms, total_bedrooms, population** och **households** var mycket starkt korrelerade med varandra, ofta med korrelationskoefficienter över 0.9. Så höga värden innebär att variablerna i praktiken fångar samma underliggande fenomen, vilket också är logiskt: antalet rum och antalet sovrum hänger naturligt samman, och både total_rooms och population tenderar att öka i områden där antalet hushåll är större. Denna typ av överlapp gör variablerna överflödiga och kan dessutom skapa multikollinearitet, vilket försämrar modellens stabilitet och tolkbarhet.  

För att undvika detta valde jag att endast behålla **households**, som på ett mer sammanfattande och stabilt sätt representerar samma struktur. De övriga tre variablerna togs därför bort ur modellen.

Variablerna longitude och latitude uppvisade också en stark inbördes korrelation, men eftersom de representerar två olika geografiska dimensioner och båda är relevanta för bostadspriser valde jag att behålla dem. Värt att notera är att den är starkt negativ på -0.92, vilket har att göra med hur det aktuella geografiska området är orienterat i koordinatsystemet.
Resultatet blev en första modell som enbart använder numeriska variabler och samtidigt undviker multikollinearitet.


In [20]:
df1 = df.drop(columns=['total_rooms', 'total_bedrooms', 'population', 'ocean_proximity'])
df1.head()

,longitude,latitude,housing_median_age,households,median_income,median_house_value
0,-122.23,37.88,41.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,259.0,3.8462,342200.0


In [21]:
X1 = df1.iloc[:, :5]
y = df1['median_house_value']
param_names_1 = df1.columns.tolist()
model_1 = LinearRegression()
model_1.fit(X1, y)
n = model.n
d = model.d
sum_mod1 = model_1.summary_basic('Model 1')
sum_coef_1 = model_1.summary_coef(param_names_1)
display(sum_mod1)
display(sum_coef_1)

,n,d,R2,Variance,Standard deviation,rmse,F-statistic,P-value
Model 1 summary,20433,5,0.6001,5.329792e+09,73005.42,73005.42,6131.3,0.0


,Parameter,Beta,Std.Err,t,p,CI lower,CI upper
0,longitude,-3659305.86,64005.58,-57.17,0.0,-3784761.92,-3533849.79
1,latitude,-43357.73,726.64,-59.67,0.0,-44782.01,-41933.45
2,housing_median_age,-42472.63,680.93,-62.37,0.0,-43807.30,-41137.96
3,households,1218.52,45.24,26.94,0.0,1129.85,1307.19
4,median_income,25.00,1.42,17.64,0.0,22.22,27.78
5,median_house_value,38209.68,283.25,134.90,0.0,37654.48,38764.87


## Andra modellen & One-hot encoding

*Manuellt skapande av one hot encoding för den kategoriska variabeln.*

Jag använder en baseline‑kategori i one‑hot‑encoding för att undvika perfekt multikollinearitet. Om alla dummy‑variabler inkluderas samtidigt blir en av dem en exakt linjär kombination av de andra, vilket gör att regressionsmodellen inte kan beräkna koefficienterna. Genom att ta bort en kategori (baseline) blir modellen identifierbar och koefficienterna kan tolkas som skillnader gentemot baseline.

In [22]:
unique_cats = df['ocean_proximity'].unique().tolist()
cats_to_encode = [cat for cat in unique_cats if cat != '<1H OCEAN']

for cat in cats_to_encode:
    df[f'{cat}'] = (df['ocean_proximity'] == cat).astype(int)

df = df.drop(columns='ocean_proximity')
df_ocean = df[['NEAR BAY', 'INLAND', 'NEAR OCEAN']]
df2 = df.drop(columns=['population', 'total_rooms', 'total_bedrooms'])
df2.head()

,longitude,latitude,housing_median_age,households,median_income,median_house_value,NEAR BAY,INLAND,NEAR OCEAN
0,-122.23,37.88,41.0,126.0,8.3252,452600.0,1,0,0
1,-122.22,37.86,21.0,1138.0,8.3014,358500.0,1,0,0
2,-122.24,37.85,52.0,177.0,7.2574,352100.0,1,0,0
3,-122.25,37.85,52.0,219.0,5.6431,341300.0,1,0,0
4,-122.25,37.85,52.0,259.0,3.8462,342200.0,1,0,0


In [23]:
X2 = df2.drop(columns=['median_house_value'])
y = df2['median_house_value']
param_names_2 = df2.columns.tolist()

model_2 = LinearRegression()
model_2.fit(X2, y)
sum_mod2 = model_2.summary_basic('Model 2')
sum_coef2 = model_2.summary_coef(param_names_2)
display(sum_mod2)
display(sum_coef2)

,n,d,R2,Variance,Standard deviation,rmse,F-statistic,P-value
Model 2 summary,20433,8,0.6128,5.161005e+09,71840.13,71840.13,4041.27,0.0


,Parameter,Beta,Std.Err,t,p,CI lower,CI upper
0,longitude,-2018330.07,90658.51,-22.26,0.00,-2196028.02,-1840632.12
1,latitude,-23692.46,1050.20,-22.56,0.00,-25750.94,-21633.99
2,housing_median_age,-22019.80,1035.88,-21.26,0.00,-24050.21,-19989.39
3,households,1093.45,45.65,23.95,0.00,1003.97,1182.92
4,median_income,24.15,1.40,17.26,0.00,21.41,26.89
5,median_house_value,37648.01,280.75,134.10,0.00,37097.73,38198.30
6,NEAR BAY,3065.32,1991.48,1.54,0.12,-838.13,6968.77
7,INLAND,-43655.11,1808.90,-24.13,0.00,-47200.70,-40109.52
8,NEAR OCEAN,10347.07,1629.37,6.35,0.00,7153.38,13540.76


## Modell 3 - endast de geografiska parametrarna

Av nyfikenhet valde jag slutligen att undersöka hur stor del av variationen i bostadspriserna som kan förklaras enbart av de geografiska variablerna (longitude, latitude och ocean‑proximity). Jag förväntade mig ett betydligt sämre resultat, dels eftersom modellen använder färre variabler, men framför allt eftersom median_income inte ingår. Median_income är den enskilt starkaste prediktorn i hela datasetet och står för en mycket stor del av modellens förklaringsgrad.

In [25]:
df3 = df.drop(columns= ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'])
X3 = df3.drop(columns= ['median_house_value'])
y = df3['median_house_value']
param_names_3 = df3.columns.tolist()
model_3 = LinearRegression()
model_3.fit(X3,y)
sum_mod3 = model_3.summary_basic('Model_3')

comparison = pd.concat([sum_mod1, sum_mod2, sum_mod3])
sum_coef3 = model_3.summary_coef(param_names_3)
display(comparison)
display(sum_coef3)

,n,d,R2,Variance,Standard deviation,rmse,F-statistic,P-value
Model 1 summary,20433,5,0.6001,5.329792e+09,73005.42,73005.42,6131.30,0.0
Model 2 summary,20433,8,0.6128,5.161005e+09,71840.13,71840.13,4041.27,0.0
Model_3 summary,20433,5,0.2697,9.734113e+09,98661.61,98661.61,1508.63,0.0


,Parameter,Beta,Std.Err,t,p,CI lower,CI upper
0,longitude,-3439288.34,123308.83,-27.89,0.00,-3680983.53,-3197593.15
1,latitude,-42574.27,1422.70,-29.93,0.00,-45362.88,-39785.67
2,median_house_value,-39939.49,1403.55,-28.46,0.00,-42690.55,-37188.43
3,NEAR BAY,3157.85,2692.50,1.17,0.24,-2119.66,8435.36
4,INLAND,-66314.15,2467.18,-26.88,0.00,-71150.01,-61478.29
5,NEAR OCEAN,-4485.60,2231.83,-2.01,0.04,-8860.16,-111.03


## Sammanfattning

Vid jämförelse av de tre modellerna framträder flera tydliga skillnader i både prediktionsförmåga och förklaringsgrad. När ocean‑variablerna inkluderas i Modell 2 förbättras modellens precision: RMSE sjunker från 72 995 till 71 824, vilket innebär att Modell 2 gör något mer träffsäkra prediktioner än Modell 1. Även R² ökar från 0.6001 till 0.6128, vilket visar att ocean‑variablerna tillför ytterligare förklaringskraft och hjälper modellen att fånga geografiska prisvariationer som inte täcks av de numeriska variablerna.  

F‑statistiken är högre i Modell 1, men detta ska nog inte tolkas som att Modell 1 är bättre. F‑värdet påverkas av antalet variabler och residual degrees of freedom, och tenderar därför att sjunka när fler variabler inkluderas. Det centrala är att båda modellerna har p‑värden nära noll, vilket innebär att de är statistiskt signifikanta som helhet. Sammantaget presterar Modell 2 bäst, både vad gäller prediktionsprecision och förklaringsgrad, vilket visar att ocean‑variablerna bidrar med relevant information om bostadspriserna.  

En intressant detalj i både modell 2 & 3 är att parametern 'NEAR BAY' har p-värden på 0.12 samt 0.24 (och även låga t-värden), vilket är över signifikansnivån 0,05. Det innebär att variabeln i dessa modeller kan klassas som ej signifikant. 

Avslutningsvis agerar modell 3 främst som ett explorativt test för att undersöka hur mycket av variationen som kan förklaras enbart av geografiskt läge. Resultatet visar att modellen endast uppnår ett R² på cirka 0.27, vilket är avsevärt lägre än i de två tidigare modellerna. Detta är förväntat eftersom flera av de mest betydelsefulla variablerna, framför allt *median_income*, inte ingår. Modell 3 illustrerar därför tydligt att geografiska faktorer visserligen har en viss förklaringskraft, men att de socioekonomiska variablerna är avgörande för att uppnå en hög prediktionsförmåga.
